In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import collections
import matplotlib.pyplot as plt
import seaborn as sns

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, TimeDistributed

from sklearn.metrics import roc_auc_score, roc_curve, auc

import rnn1

In [ ]:
def flexroc(fname):
    
    """
    
    """
    
    FLEXROC = '/home/yhuang10/Spatio-Temporal/cynet_/bin/flexroc '
    
    RES = {}
    
    h = np.array(fname.split('/')[-1].split('_')[0].split('#')).astype(float)
    RES['lat1'], RES['lat2'], RES['lon1'], RES['lon2'] = h 
    df = pd.read_csv(fname)

    df[['AR_grt','AR_prd']].to_csv('tmp.csv', header=None, index=None, sep=' ')

    result = ! $FLEXROC -i tmp.csv -w 1 -x 0 -t 0.8 -f 0.2 -E 0 -C 1 -L 1
    RES['VAR'] = float(result[0].split()[1])
    
    df[['AS_grt','AS_prd']].to_csv('tmp.csv', header=None, index=None, sep=' ')
    result = ! $FLEXROC -i tmp.csv -w 1 -x 0 -t 0.8 -f 0.2 -E 0 -C 1 -L 1
    RES['HOMICIDE-ASSAULT-BATTERY'] = float(result[0].split()[1])
    
    df[['PR_grt','PR_prd']].to_csv('tmp.csv',header=None,index=None,sep=' ')
    result = ! $FLEXROC -i tmp.csv -w 1 -x 0 -t 0.8 -f 0.2 -E 0 -C 1 -L 1
    RES['BURGLARY-THEFT-MOTOR_VEHICLE_THEFT'] = float(result[0].split()[1])
    
    ! rm tmp.csv
    
    return RES

RES = flexroc('./results/random_100-10_100/41.73216#41.73492#-87.55528000000002#-87.55176_1.rnnres') 
RES

In [2]:
#meta = pd.read_csv('meta.csv', index_col=0)
meta = pd.read_csv('meta2.csv', index_col=0)
#X_raw = np.genfromtxt('CRIME-_2014-01-01_2016-12-31.csv')
X_raw = np.genfromtxt('../time_series_covid/data.csv')


In [3]:
meta

,CASE,lat1,lat1.1,lon1,lon2
31.82571#117.2264#CASE,1,31.82571,31.82571,117.22640,117.22640
40.18238#116.4142#CASE,2,40.18238,40.18238,116.41420,116.41420
30.05718#107.874#CASE,3,30.05718,30.05718,107.87400,107.87400
26.07783#117.9895#CASE,4,26.07783,26.07783,117.98950,117.98950
36.0611#103.8343#CASE,5,36.06110,36.06110,103.83430,103.83430
23.33841#113.422#CASE,6,23.33841,23.33841,113.42200,113.42200
23.82908#108.7881#CASE,7,23.82908,23.82908,108.78810,108.78810
26.81536#106.8748#CASE,8,26.81536,26.81536,106.87480,106.87480
19.19673#109.7455#CASE,9,19.19673,19.19673,109.74550,109.74550
38.0428#114.5149#CASE,10,38.04280,38.04280,114.51490,114.51490


In [4]:
cynetTop = meta.index.values

In [5]:
#cynetTop = [
#    '41.8828#-87.7435', '41.7584#-87.6555', 
#    '41.8828#-87.6274', '41.7750#-87.6520', 
#    '41.9104#-87.7048', '41.7446#-87.6520',
#    '41.9270#-87.7364', '41.7833#-87.5957', 
#    '41.9823#-87.6555', '41.8275#-87.6168'
#]

## An example

In [6]:
center = cynetTop[0]
tile = meta.loc[center, ['lat1', 'lat2','lon1', 'lon2']]
    
future = 3
    
X_train, Y_train, X_test, Y_test = rnn1.getData(meta, X_raw, center, future)

model = rnn1.train_3(X_train, Y_train, epochs=20)
prediction = model.predict(X_test)

/home/ishanu/.local/lib/python2.7/site-packages/pandas/core/indexing.py:1017: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return getattr(section, self.name)[new_key]


Training data: input dim = (1, 27, 74), output dim = (1, 27, 1)
Out-sample data: input dim = (1, 15, 74), output dim = (1, 15, 1)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, None, 50)          25000     
_________________________________________________________________
lstm_1 (LSTM)                (None, None, 10)          2440      
_________________________________________________________________
lstm_2 (LSTM)                (None, None, 10)          840       
_________________________________________________________________
time_distributed (TimeDistri (None, None, 1)           11        
Total params: 28,291
Trainable params: 28,291
Non-trainable params: 0
_________________________________________________________________
Epoch 1/20
1/1 [==============================] - 2s 2s/step - loss: 23001.5039
Epoch 2/20
1/1 [==============================] - 0s 55ms/st

In [ ]:
figTitle = '#'.join(map(str, tile.values)) + '_{}'.format(future)
dfName = figTitle + '.rnnres'

In [21]:
Y_test=np.squeeze(Y_test)
prediction=np.squeeze(prediction)

In [22]:
from sklearn.metrics import roc_auc_score
np.corrcoef(Y_test, prediction)

array([[1.        , 0.89328198],
       [0.89328198, 1.        ]])

## Top 10 comparison:
Test the performance of (deep) RNN with horizon=$7$ on the ten blocks
on which cynet got the higest average AUC scores for the 3 variables.

In [ ]:
future = 7
RES = []
for center in cynetTop:
    tile = meta.loc[center, ['lat1', 'lat2','lon1', 'lon2']]

    X_train, Y_train, X_test, Y_test = rnn.getData(meta, X_raw, center, future)

    model = rnn.train_3(X_train, Y_train, epochs=20)
    prediction = model.predict(X_test)
    
    figTitle = '#'.join(map(str, tile.values)) + '_{}'.format(future)
    dfName = 'results/top10_deeper/' + figTitle + '.rnnres'
    rnn.Analysis(Y_test, prediction, figTitle, dfName)
    rnnres = flexroc(dfName)
    RES.append(rnnres)

In [ ]:
cynet = pd.read_csv('../data/cynet_performance.csv').round(4)
rnn = pd.DataFrame(data=RES)
rnn = rnn.set_index(['lat1','lat2','lon1','lon2']).stack().reset_index().rename(columns={'level_4':'var',0:'auc_NN'}).round(4)
df = cynet.set_index(['lat1','lat2','lon1','lon2','var']).join(rnn.set_index(['lat1','lat2','lon1','lon2','var'])).reset_index().dropna()
df

In [ ]:
sns.distplot(df.auc)
sns.distplot(df.auc_NN)
plt.show()

## Average Performance:
Test the performance of (deep) RNN with horizon=$7$ on the $50$ blocks.

In [ ]:
random = [ 
    298, 304, 908, 1176, 453,  
    281, 947, 203, 1443, 159,  
    870, 109, 103, 1206, 1084, 
    615, 1013, 324, 470, 901,
    1025, 260, 1106, 551, 82, 
    903, 1137, 1477, 210, 388,
    824, 270,  595, 269, 454,
    291, 637, 930, 292, 360,
    832, 29, 320, 498, 1181, 
    1352, 732, 1164, 488, 1124
]